In [1]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [2]:
import sys
!{sys.executable} -m pip show ace_tools_open
!{sys.executable} -m pip cache purge
# !{sys.executable} -m pip uninstall ace_tools


# !python -m pip uninstall ace_tools

Name: ace_tools_open
Version: 0.1.0
Summary: Open implementation of ace_tools referenced in GPT4o
Home-page: https://github.com/zinccat/ace_tools_open
Author: ZincCat
Author-email: ZincCat <zincchloride@outlook.com>
License: MIT
Location: /storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/lib/python3.12/site-packages
Requires: IPython, itables, pandas
Required-by: 
Files removed: 0


In [3]:
# If path to ace_tools is not listed in the sys.path you should append it manually: 

import sys
print(sys.path)

# sys.path.append("work/anaconda3/envs/volvopennstate-env/lib/python3.12/site-packages")

print(sys.executable)

!{sys.executable} --version



['/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/lib/python312.zip', '/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/lib/python3.12', '/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/lib/python3.12/lib-dynload', '', '/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/lib/python3.12/site-packages', '/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/lib/python3.12/site-packages/setuptools/_vendor']
/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin/python3.12
Python 3.12.4


In [ ]:
import findspark
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row 
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

import time as t 
from datetime import date, datetime, timedelta
from joblib import Parallel, delayed
import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys
import ace_tools_open as tools

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
from IPython import get_ipython


findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

conf = (SparkConf().set("spark.driver.maxResultSize", "4g"))

# Create new context
sc = SparkContext(conf=conf)


# sc = SparkContext("local", "Simple App")

# Create SparkSession 
spark = SparkSession.builder \
       .master("local[2]") \
       .appName("test") \
       .config("spark.driver.maxResultSize", "20g")\
       .config("spark.driver.memory", "100g")\
       .getOrCreate()

spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")

df_population = spark.sql("SELECT * FROM population")

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/30 15:50:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
calculated_features = [    
          "calendar_day", 
          "f_1_dtc38_1th_15d", 
          "f_1_dtc38_2nd_15d", 
          "f_2_dtc38_1th_15d", 
          "f_2_dtc38_2nd_15d", 
          "f_3_dtc38_1th_15d", 
          "f_3_dtc38_2nd_15d", 
          "f_4_dtc38_1th_15d", 
          "f_4_dtc38_2nd_15d", 
          "f_5_dtc38_1th_15d", 
          "f_5_dtc38_2nd_15d", 
          "f_6_dtc38_1th_15d", 
          "f_6_dtc38_2nd_15d", 
          "f_7_dtc38_1th_15d", 
          "f_7_dtc38_2nd_15d", 
          "f_8_dtc38_1th_15d", 
          "f_8_dtc38_2nd_15d", 

          "f_1_dtc75_1th_15d", 
          "f_1_dtc75_2nd_15d", 
          "f_2_dtc75_1th_15d", 
          "f_2_dtc75_2nd_15d", 
          "f_3_dtc75_1th_15d", 
          "f_3_dtc75_2nd_15d", 
          "f_4_dtc75_1th_15d", 
          "f_4_dtc75_2nd_15d", 
          "f_5_dtc75_1th_15d", 
          "f_5_dtc75_2nd_15d", 
          "f_6_dtc75_1th_15d", 
          "f_6_dtc75_2nd_15d", 
          "f_7_dtc75_1th_15d", 
          "f_7_dtc75_2nd_15d", 
          "f_8_dtc75_1th_15d", 
          "f_8_dtc75_2nd_15d",


          "f_1_dtc77_1th_15d", 
          "f_1_dtc77_2nd_15d", 
          "f_2_dtc77_1th_15d", 
          "f_2_dtc77_2nd_15d", 
          "f_3_dtc77_1th_15d", 
          "f_3_dtc77_2nd_15d", 
          "f_4_dtc77_1th_15d", 
          "f_4_dtc77_2nd_15d", 
          "f_5_dtc77_1th_15d", 
          "f_5_dtc77_2nd_15d", 
          "f_6_dtc77_1th_15d", 
          "f_6_dtc77_2nd_15d", 
          "f_7_dtc77_1th_15d", 
          "f_7_dtc77_2nd_15d", 
          "f_8_dtc77_1th_15d", 
          "f_8_dtc77_2nd_15d",


          "f_1_dtc86_1th_15d", 
          "f_1_dtc86_2nd_15d", 
          "f_2_dtc86_1th_15d", 
          "f_2_dtc86_2nd_15d", 
          "f_3_dtc86_1th_15d", 
          "f_3_dtc86_2nd_15d", 
          "f_4_dtc86_1th_15d", 
          "f_4_dtc86_2nd_15d", 
          "f_5_dtc86_1th_15d", 
          "f_5_dtc86_2nd_15d", 
          "f_6_dtc86_1th_15d", 
          "f_6_dtc86_2nd_15d", 
          "f_7_dtc86_1th_15d", 
          "f_7_dtc86_2nd_15d", 
          "f_8_dtc86_1th_15d", 
          "f_8_dtc86_2nd_15d",


          "f_1_dtc92_1th_15d", 
          "f_1_dtc92_2nd_15d", 
          "f_2_dtc92_1th_15d", 
          "f_2_dtc92_2nd_15d", 
          "f_3_dtc92_1th_15d", 
          "f_3_dtc92_2nd_15d", 
          "f_4_dtc92_1th_15d", 
          "f_4_dtc92_2nd_15d", 
          "f_5_dtc92_1th_15d", 
          "f_5_dtc92_2nd_15d", 
          "f_6_dtc92_1th_15d", 
          "f_6_dtc92_2nd_15d", 
          "f_7_dtc92_1th_15d", 
          "f_7_dtc92_2nd_15d", 
          "f_8_dtc92_1th_15d", 
          "f_8_dtc92_2nd_15d",


          "f_1_dtc94_1th_15d", 
          "f_1_dtc94_2nd_15d", 
          "f_2_dtc94_1th_15d", 
          "f_2_dtc94_2nd_15d", 
          "f_3_dtc94_1th_15d", 
          "f_3_dtc94_2nd_15d", 
          "f_4_dtc94_1th_15d", 
          "f_4_dtc94_2nd_15d", 
          "f_5_dtc94_1th_15d", 
          "f_5_dtc94_2nd_15d", 
          "f_6_dtc94_1th_15d", 
          "f_6_dtc94_2nd_15d", 
          "f_7_dtc94_1th_15d", 
          "f_7_dtc94_2nd_15d", 
          "f_8_dtc94_1th_15d", 
          "f_8_dtc94_2nd_15d",


          "f_1_dtc0401_1th_15d", 
          "f_1_dtc0401_2nd_15d", 
          "f_2_dtc0401_1th_15d", 
          "f_2_dtc0401_2nd_15d", 
          "f_3_dtc0401_1th_15d", 
          "f_3_dtc0401_2nd_15d", 
          "f_4_dtc0401_1th_15d", 
          "f_4_dtc0401_2nd_15d", 
          "f_5_dtc0401_1th_15d", 
          "f_5_dtc0401_2nd_15d", 
          "f_6_dtc0401_1th_15d", 
          "f_6_dtc0401_2nd_15d", 
          "f_7_dtc0401_1th_15d", 
          "f_7_dtc0401_2nd_15d", 
          "f_8_dtc0401_1th_15d", 
          "f_8_dtc0401_2nd_15d",


          "f_1_dtc2457_1th_15d", 
          "f_1_dtc2457_2nd_15d", 
          "f_2_dtc2457_1th_15d", 
          "f_2_dtc2457_2nd_15d", 
          "f_3_dtc2457_1th_15d", 
          "f_3_dtc2457_2nd_15d", 
          "f_4_dtc2457_1th_15d", 
          "f_4_dtc2457_2nd_15d", 
          "f_5_dtc2457_1th_15d", 
          "f_5_dtc2457_2nd_15d", 
          "f_6_dtc2457_1th_15d", 
          "f_6_dtc2457_2nd_15d", 
          "f_7_dtc2457_1th_15d", 
          "f_7_dtc2457_2nd_15d", 
          "f_8_dtc2457_1th_15d", 
          "f_8_dtc2457_2nd_15d",

          "if_parts_replaced_in_1th_15d", 
          "if_parts_replaced_in_2nd_15d"]

In [ ]:
calculated_features_minus_calendar_day = [     
          "f_1_dtc38_1th_15d", 
          "f_1_dtc38_2nd_15d", 
          "f_2_dtc38_1th_15d", 
          "f_2_dtc38_2nd_15d", 
          "f_3_dtc38_1th_15d", 
          "f_3_dtc38_2nd_15d", 
          "f_4_dtc38_1th_15d", 
          "f_4_dtc38_2nd_15d", 
          "f_5_dtc38_1th_15d", 
          "f_5_dtc38_2nd_15d", 
          "f_6_dtc38_1th_15d", 
          "f_6_dtc38_2nd_15d", 
          "f_7_dtc38_1th_15d", 
          "f_7_dtc38_2nd_15d", 
          "f_8_dtc38_1th_15d", 
          "f_8_dtc38_2nd_15d", 

          "f_1_dtc75_1th_15d", 
          "f_1_dtc75_2nd_15d", 
          "f_2_dtc75_1th_15d", 
          "f_2_dtc75_2nd_15d", 
          "f_3_dtc75_1th_15d", 
          "f_3_dtc75_2nd_15d", 
          "f_4_dtc75_1th_15d", 
          "f_4_dtc75_2nd_15d", 
          "f_5_dtc75_1th_15d", 
          "f_5_dtc75_2nd_15d", 
          "f_6_dtc75_1th_15d", 
          "f_6_dtc75_2nd_15d", 
          "f_7_dtc75_1th_15d", 
          "f_7_dtc75_2nd_15d", 
          "f_8_dtc75_1th_15d", 
          "f_8_dtc75_2nd_15d",


          "f_1_dtc77_1th_15d", 
          "f_1_dtc77_2nd_15d", 
          "f_2_dtc77_1th_15d", 
          "f_2_dtc77_2nd_15d", 
          "f_3_dtc77_1th_15d", 
          "f_3_dtc77_2nd_15d", 
          "f_4_dtc77_1th_15d", 
          "f_4_dtc77_2nd_15d", 
          "f_5_dtc77_1th_15d", 
          "f_5_dtc77_2nd_15d", 
          "f_6_dtc77_1th_15d", 
          "f_6_dtc77_2nd_15d", 
          "f_7_dtc77_1th_15d", 
          "f_7_dtc77_2nd_15d", 
          "f_8_dtc77_1th_15d", 
          "f_8_dtc77_2nd_15d",


          "f_1_dtc86_1th_15d", 
          "f_1_dtc86_2nd_15d", 
          "f_2_dtc86_1th_15d", 
          "f_2_dtc86_2nd_15d", 
          "f_3_dtc86_1th_15d", 
          "f_3_dtc86_2nd_15d", 
          "f_4_dtc86_1th_15d", 
          "f_4_dtc86_2nd_15d", 
          "f_5_dtc86_1th_15d", 
          "f_5_dtc86_2nd_15d", 
          "f_6_dtc86_1th_15d", 
          "f_6_dtc86_2nd_15d", 
          "f_7_dtc86_1th_15d", 
          "f_7_dtc86_2nd_15d", 
          "f_8_dtc86_1th_15d", 
          "f_8_dtc86_2nd_15d",


          "f_1_dtc92_1th_15d", 
          "f_1_dtc92_2nd_15d", 
          "f_2_dtc92_1th_15d", 
          "f_2_dtc92_2nd_15d", 
          "f_3_dtc92_1th_15d", 
          "f_3_dtc92_2nd_15d", 
          "f_4_dtc92_1th_15d", 
          "f_4_dtc92_2nd_15d", 
          "f_5_dtc92_1th_15d", 
          "f_5_dtc92_2nd_15d", 
          "f_6_dtc92_1th_15d", 
          "f_6_dtc92_2nd_15d", 
          "f_7_dtc92_1th_15d", 
          "f_7_dtc92_2nd_15d", 
          "f_8_dtc92_1th_15d", 
          "f_8_dtc92_2nd_15d",


          "f_1_dtc94_1th_15d", 
          "f_1_dtc94_2nd_15d", 
          "f_2_dtc94_1th_15d", 
          "f_2_dtc94_2nd_15d", 
          "f_3_dtc94_1th_15d", 
          "f_3_dtc94_2nd_15d", 
          "f_4_dtc94_1th_15d", 
          "f_4_dtc94_2nd_15d", 
          "f_5_dtc94_1th_15d", 
          "f_5_dtc94_2nd_15d", 
          "f_6_dtc94_1th_15d", 
          "f_6_dtc94_2nd_15d", 
          "f_7_dtc94_1th_15d", 
          "f_7_dtc94_2nd_15d", 
          "f_8_dtc94_1th_15d", 
          "f_8_dtc94_2nd_15d",


          "f_1_dtc0401_1th_15d", 
          "f_1_dtc0401_2nd_15d", 
          "f_2_dtc0401_1th_15d", 
          "f_2_dtc0401_2nd_15d", 
          "f_3_dtc0401_1th_15d", 
          "f_3_dtc0401_2nd_15d", 
          "f_4_dtc0401_1th_15d", 
          "f_4_dtc0401_2nd_15d", 
          "f_5_dtc0401_1th_15d", 
          "f_5_dtc0401_2nd_15d", 
          "f_6_dtc0401_1th_15d", 
          "f_6_dtc0401_2nd_15d", 
          "f_7_dtc0401_1th_15d", 
          "f_7_dtc0401_2nd_15d", 
          "f_8_dtc0401_1th_15d", 
          "f_8_dtc0401_2nd_15d",


          "f_1_dtc2457_1th_15d", 
          "f_1_dtc2457_2nd_15d", 
          "f_2_dtc2457_1th_15d", 
          "f_2_dtc2457_2nd_15d", 
          "f_3_dtc2457_1th_15d", 
          "f_3_dtc2457_2nd_15d", 
          "f_4_dtc2457_1th_15d", 
          "f_4_dtc2457_2nd_15d", 
          "f_5_dtc2457_1th_15d", 
          "f_5_dtc2457_2nd_15d", 
          "f_6_dtc2457_1th_15d", 
          "f_6_dtc2457_2nd_15d", 
          "f_7_dtc2457_1th_15d", 
          "f_7_dtc2457_2nd_15d", 
          "f_8_dtc2457_1th_15d", 
          "f_8_dtc2457_2nd_15d",

          "if_parts_replaced_in_1th_15d", 
          "if_parts_replaced_in_2nd_15d"]

In [ ]:
egr_cooler_claims_file_path = "/storage/home/yqf5148/work/volvoPennState/EGR Cooler Claims.csv"
df_egr_cooler_claims = pd.read_csv(egr_cooler_claims_file_path, header=None, names=['BUSINESS_UNIT', 'CAMP_TYPE', 'CLAIM_HANDLING_DECISION', 'CLAIM_JOB_ID', 'CLAIM_REG_DATE', 'CLAIM_STATUS_DETAIL', 'CRED_DATE', 'DEB_CD', 'DEB_CD_DESC', 'DES_AREA', 'FGRP4', 'FGRP4_DESC', 'MAIN_OP', 'MAIN_OP_DESC', 'MILEAGE_MILES_CLAIM', 'PART_NAME', 'PART_NO', 'REF_NO', 'REPAIR_ID', 'REP_CNTRY', 'REP_DATE', 'REP_DEALER', 'REP_DEALER_NAME', 'REP_REGION', 'REP_STATE', 'SUPPL_NAME', 'SUPPL_NO', 'TOT_CLAIM_PAYMENT_USD', 'TOT_LABOR_PAYMENT_USD', 'TOT_MTR_PAYMENT_USD', 'TOT_OTHER_PAYMENT_USD', 'TOT_TRAVEL_PAYMENT_USD', 'VEH_AGE_CLAIM', 'VMT', 'WIS_CLAIM_ID', 'WIS_PRODUCT_ID', 'VEHICLE_ASEMBLY_DATE', 'APPL_TYPE', 'BRAND', 'CHASSIS_SERIES', 'CHASSIS_NO', 'COMPANY', 'CUST_NAME', 'CUST_NO', 'DEL_CNTRY', 'DEL_DEALER', 'DEL_DEALER_NAME', 'DEL_REGION', 'DEL_STATE', 'DEL_STATUS', 'ENGINE_ASSEMB_DATE', 'ENGINE_HP', 'ENGINE_MK', 'ENGINE_SIZE', 'INS_DATE', 'LAST_KNOWN_MILEAGE_MILES', 'SN_ENGINE', 'SN_GEARB', 'VEH_AGE', 'VEH_ASSEMB_DATE', 'VEH_ASSEMB_YEAR', 'VEH_MD_FAMILY', 'VEH_TYPE', 'VIN', 'COM_FIELD1', 'COM_FIELD2', 'COM_FIELD3', 'COM_FIELD_4_EXTERNAL'], index_col=False, encoding='ISO-8859-1', dtype='unicode')

egr_sensors_claim_file_path = "/storage/home/yqf5148/work/volvoPennState/EGR Sensors.csv"
df_egr_sensors_claims = pd.read_csv(egr_cooler_claims_file_path, header=None, names=['BUSINESS_UNIT', 'CAMP_TYPE', 'CLAIM_HANDLING_DECISION', 'CLAIM_JOB_ID', 'CLAIM_REG_DATE', 'CLAIM_STATUS_DETAIL', 'CRED_DATE', 'DEB_CD', 'DEB_CD_DESC', 'DES_AREA', 'FGRP4', 'FGRP4_DESC', 'MAIN_OP', 'MAIN_OP_DESC', 'MILEAGE_MILES_CLAIM', 'PART_NAME', 'PART_NO', 'REF_NO', 'REPAIR_ID', 'REP_CNTRY', 'REP_DATE', 'REP_DEALER', 'REP_DEALER_NAME', 'REP_REGION', 'REP_STATE', 'SUPPL_NAME', 'SUPPL_NO', 'TOT_CLAIM_PAYMENT_USD', 'TOT_LABOR_PAYMENT_USD', 'TOT_MTR_PAYMENT_USD', 'TOT_OTHER_PAYMENT_USD', 'TOT_TRAVEL_PAYMENT_USD', 'VEH_AGE_CLAIM', 'VMT', 'WIS_CLAIM_ID', 'WIS_PRODUCT_ID', 'VEHICLE_ASEMBLY_DATE', 'APPL_TYPE', 'BRAND', 'CHASSIS_SERIES', 'CHASSIS_NO', 'COMPANY', 'CUST_NAME', 'CUST_NO', 'DEL_CNTRY', 'DEL_DEALER', 'DEL_DEALER_NAME', 'DEL_REGION', 'DEL_STATE', 'DEL_STATUS', 'ENGINE_ASSEMB_DATE', 'ENGINE_HP', 'ENGINE_MK', 'ENGINE_SIZE', 'INS_DATE', 'LAST_KNOWN_MILEAGE_MILES', 'SN_ENGINE', 'SN_GEARB', 'VEH_AGE', 'VEH_ASSEMB_DATE', 'VEH_ASSEMB_YEAR', 'VEH_MD_FAMILY', 'VEH_TYPE', 'VIN', 'COM_FIELD1', 'COM_FIELD2', 'COM_FIELD3', 'COM_FIELD_4_EXTERNAL'], index_col=False, encoding='ISO-8859-1', dtype='unicode')

egr_fg_293_claims_file_path = "/storage/home/yqf5148/work/volvoPennState/EGR FG 293 Claims.csv"
df_egr_fg_293_claims = pd.read_csv(egr_cooler_claims_file_path, header=None, names=['BUSINESS_UNIT', 'CAMP_TYPE', 'CLAIM_HANDLING_DECISION', 'CLAIM_JOB_ID', 'CLAIM_REG_DATE', 'CLAIM_STATUS_DETAIL', 'CRED_DATE', 'DEB_CD', 'DEB_CD_DESC', 'DES_AREA', 'FGRP4', 'FGRP4_DESC', 'MAIN_OP', 'MAIN_OP_DESC', 'MILEAGE_MILES_CLAIM', 'PART_NAME', 'PART_NO', 'REF_NO', 'REPAIR_ID', 'REP_CNTRY', 'REP_DATE', 'REP_DEALER', 'REP_DEALER_NAME', 'REP_REGION', 'REP_STATE', 'SUPPL_NAME', 'SUPPL_NO', 'TOT_CLAIM_PAYMENT_USD', 'TOT_LABOR_PAYMENT_USD', 'TOT_MTR_PAYMENT_USD', 'TOT_OTHER_PAYMENT_USD', 'TOT_TRAVEL_PAYMENT_USD', 'VEH_AGE_CLAIM', 'VMT', 'WIS_CLAIM_ID', 'WIS_PRODUCT_ID', 'VEHICLE_ASEMBLY_DATE', 'APPL_TYPE', 'BRAND', 'CHASSIS_SERIES', 'CHASSIS_NO', 'COMPANY', 'CUST_NAME', 'CUST_NO', 'DEL_CNTRY', 'DEL_DEALER', 'DEL_DEALER_NAME', 'DEL_REGION', 'DEL_STATE', 'DEL_STATUS', 'ENGINE_ASSEMB_DATE', 'ENGINE_HP', 'ENGINE_MK', 'ENGINE_SIZE', 'INS_DATE', 'LAST_KNOWN_MILEAGE_MILES', 'SN_ENGINE', 'SN_GEARB', 'VEH_AGE', 'VEH_ASSEMB_DATE', 'VEH_ASSEMB_YEAR', 'VEH_MD_FAMILY', 'VEH_TYPE', 'VIN', 'COM_FIELD1', 'COM_FIELD2', 'COM_FIELD3', 'COM_FIELD_4_EXTERNAL'], index_col=False, encoding='ISO-8859-1', dtype='unicode')

# Finding the claims, type of them and filter them based on part_replacements for VINs in train data 

In [ ]:
# Convert the TOT_CLAIM_PAYMENT_USD column to numeric, forcing errors to NaN (in case of any non-numeric values)
df_egr_cooler_claims['TOT_CLAIM_PAYMENT_USD'] = pd.to_numeric(df_egr_cooler_claims['TOT_CLAIM_PAYMENT_USD'], errors='coerce')
df_egr_fg_293_claims['TOT_CLAIM_PAYMENT_USD'] = pd.to_numeric(df_egr_fg_293_claims['TOT_CLAIM_PAYMENT_USD'], errors='coerce')
df_egr_sensors_claims['TOT_CLAIM_PAYMENT_USD'] = pd.to_numeric(df_egr_sensors_claims['TOT_CLAIM_PAYMENT_USD'], errors='coerce')


# Load the clustered_aggregated_train_df data
columns_names = ['VIN', 'ENGINE_SIZE', 'ENGINE_HP', 'VEH_TYPE'] + [s for s in df_population.columns if 'KOLA' in s] + calculated_features_minus_calendar_day + ['CLUSTER']
clustered_aggregated_train_df = pd.read_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/clustered_aggregated_train_df.csv', sep=',', names=columns_names, header=None)


# Filter only those claims that match VINs in clustered_aggregated_train_df
claims_matches_cooler = df_egr_cooler_claims[df_egr_cooler_claims['VIN'].isin(clustered_aggregated_train_df.iloc[:, 0])]
claims_matches_fg = df_egr_fg_293_claims[df_egr_fg_293_claims['VIN'].isin(clustered_aggregated_train_df.iloc[:, 0])]
claims_matches_sensors = df_egr_sensors_claims[df_egr_sensors_claims['VIN'].isin(clustered_aggregated_train_df.iloc[:, 0])]


# Add a CLAIM_TYPE column
claims_matches_cooler['CLAIM_TYPE'] = 'EGR_COOLER'
claims_matches_fg['CLAIM_TYPE'] = 'EGR_FG'
claims_matches_sensors['CLAIM_TYPE'] = 'EGR_SENSORS'


# Add the PART_REPL column (1 if TOT_CLAIM_PAYMENT_USD > 1000, else 0)
for df in [claims_matches_cooler, claims_matches_fg, claims_matches_sensors]:
    df['PART_REPL'] = df['TOT_CLAIM_PAYMENT_USD'] > 1000.0
    df['PART_REPL'] = df['PART_REPL'].astype(int)

    
# Merge with clustered_aggregated_train_df to get CLUSTER value
claims_matches_cooler = pd.merge(claims_matches_cooler, clustered_aggregated_train_df[['VIN', 'CLUSTER']], on='VIN', how='left')
claims_matches_fg = pd.merge(claims_matches_fg, clustered_aggregated_train_df[['VIN', 'CLUSTER']], on='VIN', how='left')
claims_matches_sensors = pd.merge(claims_matches_sensors, clustered_aggregated_train_df[['VIN', 'CLUSTER']], on='VIN', how='left')

claims_matches = pd.concat([claims_matches_cooler, claims_matches_fg, claims_matches_sensors], ignore_index=True)


# Calculate DAY_BACK for each claim
reference_date = pd.to_datetime('2021-12-31')
claims_matches['CLAIM_REG_DATE'] = pd.to_datetime(claims_matches['CLAIM_REG_DATE'])
claims_matches['DAYS_BACK'] = (reference_date - claims_matches['CLAIM_REG_DATE']).dt.days


# Select relevant columns, including the new PART_REPL column
result_df_for_train_data = claims_matches[['VIN', 'CLAIM_JOB_ID', 'CLAIM_REG_DATE', 'TOT_CLAIM_PAYMENT_USD', 'DAYS_BACK', 'CLAIM_TYPE', 'CLUSTER', 'PART_REPL']]


# Display the resulting dataframe
print(result_df_for_train_data)


# Save the resulting dataframe to a CSV file
result_df_for_train_data.to_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/claims_result_for_train_data.csv', index=False)


# Finding the claims, type of them and filter them based on part_replacements for VINs in test data 

In [ ]:
# Load the test DataFrame
clustered_aggregated_test_df = pd.read_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/clustered_aggregated_test_df.csv', sep=',', names=columns_names, header=None)


# Filter only those claims that match VINs in clustered_aggregated_test_df
claims_matches_cooler = df_egr_cooler_claims[df_egr_cooler_claims['VIN'].isin(clustered_aggregated_test_df.iloc[:, 0])]
claims_matches_fg = df_egr_fg_293_claims[df_egr_fg_293_claims['VIN'].isin(clustered_aggregated_test_df.iloc[:, 0])]
claims_matches_sensors = df_egr_sensors_claims[df_egr_sensors_claims['VIN'].isin(clustered_aggregated_test_df.iloc[:, 0])]


# Add a CLAIM_TYPE column
claims_matches_cooler['CLAIM_TYPE'] = 'EGR_COOLER'
claims_matches_fg['CLAIM_TYPE'] = 'EGR_FG'
claims_matches_sensors['CLAIM_TYPE'] = 'EGR_SENSORS'


# Add the PART_REPL column (1 if TOT_CLAIM_PAYMENT_USD > 1000, else 0)
for df in [claims_matches_cooler, claims_matches_fg, claims_matches_sensors]:
    df['PART_REPL'] = df['TOT_CLAIM_PAYMENT_USD'] > 1000.0
    df['PART_REPL'] = df['PART_REPL'].astype(int)


# Merge with clustered_aggregated_test_df to get CLUSTER value
claims_matches_cooler = pd.merge(claims_matches_cooler, clustered_aggregated_test_df[['VIN', 'CLUSTER']], on='VIN', how='left')
claims_matches_fg = pd.merge(claims_matches_fg, clustered_aggregated_test_df[['VIN', 'CLUSTER']], on='VIN', how='left')
claims_matches_sensors = pd.merge(claims_matches_sensors, clustered_aggregated_test_df[['VIN', 'CLUSTER']], on='VIN', how='left')


claims_matches = pd.concat([claims_matches_cooler, claims_matches_fg, claims_matches_sensors], ignore_index=True)


# Calculate DAY_BACK for each claim
reference_date = pd.to_datetime('2021-12-31')
claims_matches['CLAIM_REG_DATE'] = pd.to_datetime(claims_matches['CLAIM_REG_DATE'])
claims_matches['DAYS_BACK'] = (reference_date - claims_matches['CLAIM_REG_DATE']).dt.days


# Select relevant columns, including the new PART_REPL column
result_df_for_test_data = claims_matches[['VIN', 'CLAIM_JOB_ID', 'CLAIM_REG_DATE', 'TOT_CLAIM_PAYMENT_USD', 'DAYS_BACK', 'CLAIM_TYPE', 'CLUSTER', 'PART_REPL']]


# Display the resulting dataframe
print(result_df_for_test_data)


# Save the resulting dataframe to a CSV file
result_df_for_test_data.to_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/claims_result_for_test_data.csv', index=False)


# Finding dates that are the count of clanedar_day(s) back from date 31/12/2021, for which there is a matching claim of part replacement exist and is registered for this VIN in train data.

In [ ]:
# Load the DTC file (sample for DTC 75)
cleaned_resultedData_path = '/storage/home/yqf5148/work/volvoPennState/data/dataset/cleanedNormalized_resultedData.csv'
all_columns_names = ['VIN','ENGINE_SIZE','ENGINE_HP','VEH_TYPE']+[s for s in df_population.columns if 'KOLA' in s] + calculated_features

# Read the cleaned CSV file into a DataFrame
df_resultedData = pd.read_csv(cleaned_resultedData_path, header=None, names=all_columns_names, index_col=False, dtype='unicode')

# Load the replacement claims data
df_claims_on_train_data = pd.read_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/claims_result_for_train_data.csv')

# Convert date columns to datetime
df_claims_on_train_data['CLAIM_REG_DATE'] = pd.to_datetime(df_claims_on_train_data['CLAIM_REG_DATE'])
# Convert 'CALENDAR_DAY' in resultedData to int if not already
df_claims_on_train_data['DAYS_BACK'] = df_claims_on_train_data['DAYS_BACK'].astype(int)
df_resultedData['calendar_day'] = df_resultedData['calendar_day'].astype(int)

# Initialize a list to store matching results
matching_results = []

# Iterate over each row in df_claims_on_train_data
i=0
for _, claim_row in df_claims_on_train_data.iterrows():
    vin = claim_row['VIN']
    print("VIN in training: ")
    print(vin)
    days_back = claim_row['DAYS_BACK']
    i = i+1
    print("{0}-days_back for this VIN: {1}".format(i, days_back))
    
    
    # Filter resultedData for the current VIN
    vin_matches = df_resultedData[df_resultedData['VIN'] == vin]
    # display(vin_matches)
    
    matching_vin_matches = vin_matches[vin_matches['calendar_day'] == days_back]

    if not matching_vin_matches.empty:
        matching_results.append({'VIN': vin, 'CLAIM_JOB_ID': claim_row['CLAIM_JOB_ID'], 'CLAIM_REG_DATE': claim_row['CLAIM_REG_DATE'], 'TOT_CLAIM_PAYMENT_USD': claim_row['TOT_CLAIM_PAYMENT_USD'], 'DAYS_BACK': claim_row['DAYS_BACK'], 'CLAIM_TYPE': claim_row['CLAIM_TYPE'], 'CLUSTER': claim_row['CLUSTER'], 'PART_REPL': claim_row['PART_REPL']})



# Convert the list of matching results to a DataFrame
matching_results_for_train_df = pd.DataFrame(matching_results)

# Display the matching results

tools.display_dataframe_to_user(name="Matching VIN, CLAIM_JOB_ID, DAYS_BACK and CLAIM_REG_DATE", dataframe=matching_results_for_train_df)



# Finding dates that are the count of clanedar_day(s) back from date 31/12/2021, for which there is a matching claim of part replacement exist and is registered for this VIN in test data.

In [ ]:
# Load the DTC file (sample for DTC 75)
cleaned_resultedData_path = '/storage/home/yqf5148/work/volvoPennState/data/dataset/cleanedNormalized_resultedData.csv'
all_columns_names = ['VIN','ENGINE_SIZE','ENGINE_HP','VEH_TYPE']+[s for s in df_population.columns if 'KOLA' in s] + calculated_features

# Read the cleaned CSV file into a DataFrame
df_resultedData = pd.read_csv(cleaned_resultedData_path, header=None, names=all_columns_names, index_col=False, dtype='unicode')

# Load the replacement claims data
df_claims_on_test_data = pd.read_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/claims_result_for_test_data.csv')

# Convert date columns to datetime
df_claims_on_test_data['CLAIM_REG_DATE'] = pd.to_datetime(df_claims_on_test_data['CLAIM_REG_DATE'])
# Convert 'CALENDAR_DAY' in resultedData to int if not already
df_claims_on_test_data['DAYS_BACK'] = df_claims_on_test_data['DAYS_BACK'].astype(int)

df_resultedData['calendar_day'] = df_resultedData['calendar_day'].astype(int)

# Initializ e a list to store matching results
matching_results = []
i=0
# Iterate over each row in df_claims_on_train_data
for _, claim_row in df_claims_on_test_data.iterrows():
    vin = claim_row['VIN']
    print("VIN in training: ")
    print(vin)
    days_back = claim_row['DAYS_BACK']
    i = i+1
    print("{0}-days_back for this VIN: {1}".format(i, days_back))

    
    
    # Filter resultedData for the current VIN
    vin_matches = df_resultedData[df_resultedData['VIN'] == vin]
    # display(vin_matches)
    
    matching_vin_matches = vin_matches[vin_matches['calendar_day'] == days_back]

    if not matching_vin_matches.empty:
        matching_results.append({'VIN': vin, 'CLAIM_JOB_ID': claim_row['CLAIM_JOB_ID'], 'CLAIM_REG_DATE': claim_row['CLAIM_REG_DATE'], 'TOT_CLAIM_PAYMENT_USD': claim_row['TOT_CLAIM_PAYMENT_USD'], 'DAYS_BACK': claim_row['DAYS_BACK'], 'CLAIM_TYPE': claim_row['CLAIM_TYPE'], 'CLUSTER': claim_row['CLUSTER'], 'PART_REPL': claim_row['PART_REPL']})



# Convert the list of matching results to a DataFrame
matching_results_for_test_df = pd.DataFrame(matching_results)

# Display the matching results

tools.display_dataframe_to_user(name="Matching VIN, CLAIM_JOB_ID, DAYS_BACK and CLAIM_REG_DATE", dataframe=matching_results_for_test_df)

# Explanation: for a matched row in matching_results_for_train_df, I want to look into that VIN's information in the df_resultedData for a duration of 30 days back from this matched date (calendar_day shows the certain days back from 31/12/2021. for example, if calendar_day is 10 this date is 21/12/2021 and we need to look back at the duration of 30 days from this date, which would be the time duration between dates 21/11/2021 and 21/12/2021. For this time duration, and to do the task of capturing the (degradation) pattern for faulty operation signals, we need to capture the pattern of calculated features from column f_1_dtc38_1th_15d to column f_8_dtc2457_2nd_15d (128 columns). We will then need to compare this captured pattern of these 128 signals with a similar VIN from the test_df that is in the same cluster with this VIN number that is from train_df. So, the captured 128 signals for this duration of 30 days back from that 31/12/2021 - calendar_day (31/12/2021 - 10 = 21/12/2021 in the above example) will later be used as an index for degradation pattern of monitored elements in the truck engine. (these changes of the degradation pattern are shown through the changes of these 128 signals

In [ ]:
# Step 1: Initialize a list to store degradation patterns for matched VINs
degradation_patterns = []

# Iterate over each row in matching_results_for_train_df
for _, match in matching_results_for_train_df.iterrows():
    vin = match['VIN']
    claim_reg_date = match['CLAIM_REG_DATE']
    days_back = match['DAYS_BACK']

    # Find the calendar_day corresponding to this VIN in the matched row
    calculatedFeaturesFromResultedData_vin_matches = df_resultedData[df_resultedData['VIN'] == vin]
    calendar_day_match = calculatedFeaturesFromResultedData_vin_matches[calculatedFeaturesFromResultedData_vin_matches['calendar_day'] == days_back]

    if not calendar_day_match.empty:
        # Step 2: Define the 30-day window for degradation pattern
        target_day = int(calendar_day_match.iloc[0]['calendar_day'])
        start_day = target_day + 30  # 30 days back
        end_day = target_day         # Ending on the matched day

        # Filter for the 30-day window in df_resultedData
        calculatedFeaturesFromResultedData_vin_matches_30_day_window = calculatedFeaturesFromResultedData_vin_matches[
            (calculatedFeaturesFromResultedData_vin_matches['calendar_day'] >= end_day) &
            (calculatedFeaturesFromResultedData_vin_matches['calendar_day'] <= start_day)
        ]
        
        if not calculatedFeaturesFromResultedData_vin_matches_30_day_window.empty:
            # Step 3: Define the feature columns and create the structured DataFrame
            feature_columns = ['calendar_day'] + \
                              [col for col in df_resultedData.columns if col.startswith('f_')] + \
                              [col for col in df_resultedData.columns if col.endswith('_parts')]

            # Extract the relevant columns and convert to DataFrame
            vin_degradation_pattern = calculatedFeaturesFromResultedData_vin_matches_30_day_window[feature_columns]
            vin_degradation_pattern_df = pd.DataFrame(vin_degradation_pattern.values, columns=feature_columns)

            # Store the degradation pattern along with VIN and claim details
            degradation_patterns.append({
                'VIN': vin,
                'CLAIM_JOB_ID': match['CLAIM_JOB_ID'],
                'CLAIM_REG_DATE': claim_reg_date,
                'CLAIM_TYPE': match['CLAIM_TYPE'],
                'CLUSTER': match['CLUSTER'],
                'PATTERN': vin_degradation_pattern_df  # Store as structured DataFrame
            })

# Step 4: Create a DataFrame from the collected patterns
degradation_patterns_df_from_train_data = pd.DataFrame(degradation_patterns)

# Display the patterns to the user
tools.display_dataframe_to_user(name="Degradation Patterns for 30-Day Window", dataframe=degradation_patterns_df_from_train_data)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Step 0: Load degradation patterns from degradation_patterns_df_from_train_data and identify unique clusters
clusters = degradation_patterns_df_from_train_data['CLUSTER'].unique()
models_by_cluster = {}  # Dictionary to store models for each cluster

# Step 1: Iterate over each unique cluster
for cluster in clusters:
    # Filter degradation patterns for this specific cluster
    cluster_data = degradation_patterns_df_from_train_data[degradation_patterns_df_from_train_data['CLUSTER'] == cluster]
    
    # Initialize lists to collect data from all VINs in this cluster
    X_cluster = []
    y_cluster = []
    
    # Identify unique VINs within the cluster
    vins = cluster_data['VIN'].unique()
    
    for vin in vins:
        # Filter data for this specific VIN within the cluster
        vin_data = cluster_data[cluster_data['VIN'] == vin]

        # Extract features (X) and targets (y) from the 'PATTERN' column in vin_data
        # Assuming 'PATTERN' is a DataFrame itself with feature columns (_f) and target columns (_parts)
        for _, pattern_row in vin_data.iterrows():
            vin_pattern_df = pattern_row['PATTERN']  # Extract the DataFrame stored in 'PATTERN' column

            # Separate features and target within the 30-day degradation pattern for this VIN
            X_vin = vin_pattern_df[[col for col in vin_pattern_df.columns if col.startswith('f_')]]
            y_vin = vin_pattern_df[[col for col in vin_pattern_df.columns if col.endswith('_parts')]].values.ravel()  # Flatten for model input

            # Append data to the cluster lists
            X_cluster.append(X_vin)
            y_cluster.append(y_vin)
    
    # Concatenate all VIN data within this cluster
    X_cluster = pd.concat(X_cluster)
    y_cluster = pd.concat([pd.Series(y) for y in y_cluster])  # Convert each y array to Series and concatenate

    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_cluster, y_cluster, test_size=0.2, random_state=42)

    # Initialize and train the model for this cluster
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # Optional: Evaluate model on validation set
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    print(f"Cluster {cluster} - Validation Accuracy: {accuracy:.2f}")

    # Store the trained model for this cluster
    models_by_cluster[cluster] = model

print("Training completed for all clusters.")


In [1]:
# # Step 0: Read sorted_VINs_by_similarity dataframe
# sorted_VINs_by_similarity = pd.read_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/sorted_VINs_by_similarity.csv', sep=',', header=None)

# # Step 1: Initialize a list to store comparison results
# comparison_results = []

# # Iterate over each VIN in the test set (matching_results_for_test_df)
# for _, test_vin_row in matching_results_for_test_df.iterrows():
#     vin_a = test_vin_row['VIN']
#     vin_a_cluster = test_vin_row['CLUSTER']
    
#     # Extract VIN A's degradation patterns from the 30-day window (from resultedData)
#     vin_a_degradation_patterns = df_resultedData[df_resultedData['VIN'] == vin_a]
    
#     # Retrieve the row from sorted_VINs_by_similarity where column 1 matches VIN A (VIN from test data)
#     similar_vins_row = sorted_VINs_by_similarity[sorted_VINs_by_similarity.iloc[:, 1] == vin_a]
    
#     if similar_vins_row.empty:
#         # If no similar VINs are found, skip to the next iteration
#         continue
    
#     # Extract the similar VINs (all columns from 2 onwards in the sorted_VINs_by_similarity dataframe)
#     similar_vins = similar_vins_row.iloc[0, 2:].dropna().values.tolist()  # Get list of similar VINs in descending order of similarity
#     print("Similar VINs in train data to {0} are".format(vin_a))
#     print(similar_vins)
#     # Iterate over the similar VINs (from train data)
#     for similar_vin in similar_vins:
#         # Retrieve the degradation patterns for this similar VIN
#         similar_vin_degradation_patterns = df_resultedData[df_resultedData['VIN'] == similar_vin]

#         # Iterate over each day, going back one day at a time to form 30-day windows for comparison
#         for day in range(1, len(similar_vin_degradation_patterns) - 30):
#             # Define the 30-day window for VIN A (from test data)
#             vin_a_30_day_window = vin_a_degradation_patterns[(vin_a_degradation_patterns['calendar_day'] >= day) &
#                                                             (vin_a_degradation_patterns['calendar_day'] <= day + 30)]

#             # Define the corresponding 30-day window for the similar VIN from train data
#             similar_vin_30_day_window = similar_vin_degradation_patterns[(similar_vin_degradation_patterns['calendar_day'] >= day) &
#                                                                          (similar_vin_degradation_patterns['calendar_day'] <= day + 30)]
            
#             if not vin_a_30_day_window.empty and not similar_vin_30_day_window.empty:
#                 # Step 2: Perform pattern comparison between VIN A and the similar VIN
#                 comparison_result = {
#                     'VIN_A': vin_a,
#                     'Similar_VIN': similar_vin,
#                     'Cluster': vin_a_cluster,
#                     'Day': day,
#                     'VIN_A_Pattern': vin_a_30_day_window,
#                     'Similar_VIN_Pattern': similar_vin_30_day_window,
#                 }

#                 # Append the comparison result to the list
#                 comparison_results.append(comparison_result)

# # Step 3: Store the comparison results in a dataframe for further analysis or display
# comparison_results_df = pd.DataFrame(comparison_results)

# # Display the comparison results dataframe
# tools.display_dataframe_to_user(name="VIN Comparison Results", dataframe=comparison_results_df)


In [ ]:
# from sklearn.ensemble import IsolationForest
# from sklearn.preprocessing import StandardScaler
# import joblib

# # Dictionary to store models for each cluster
# cluster_models = {}

# # Iterate over each unique cluster
# for cluster in degradation_patterns_df['CLUSTER'].unique():
#     # Filter data for the current cluster
#     cluster_data = degradation_patterns_df[degradation_patterns_df['CLUSTER'] == cluster]
    
#     # Extract degradation patterns and standardize them
#     patterns = []
#     for _, row in cluster_data.iterrows():
#         pattern = row['PATTERN']  # This should be a DataFrame or Series of features
#         patterns.append(pattern.values.flatten())  # Flattening to get feature vector

#     # Standardize patterns
#     scaler = StandardScaler()
#     patterns_scaled = scaler.fit_transform(patterns)
    
#     # Train the Isolation Forest model on scaled patterns
#     model = IsolationForest(random_state=42)
#     model.fit(patterns_scaled)
    
#     # Store the model and scaler for this cluster
#     cluster_models[cluster] = {'model': model, 'scaler': scaler}
    
#     # Optionally save the model and scaler using joblib
#     joblib.dump(model, f'cluster_{cluster}_isolation_forest_model.joblib')
#     joblib.dump(scaler, f'cluster_{cluster}_scaler.joblib')

# print("Models trained and saved for each cluster.")


In [ ]:
# # Iterate over each VIN in degradation_patterns_df
# for _, pattern_data in degradation_patterns_df.iterrows():
#     vin = pattern_data['VIN']
#     cluster = pattern_data['CLUSTER']
#     claim_type = pattern_data['CLAIM_TYPE']
#     pattern_df = pattern_data['PATTERN']
    
#     # Create a new figure for the merged plot
#     plt.figure(figsize=(12, 8))
    
#     # Iterate over each 'f_' feature column
#     feature_columns = [col for col in pattern_df.columns if col.startswith('f_')]
#     for feature in feature_columns:
#         # Plot each feature on the same axes
#         plt.plot(pattern_df['calendar_day'], pattern_df[feature], marker='o', label=feature)
    
#     # Set plot title and labels
#     plt.title(f'Degradation Patterns for All Features (VIN: {vin}, CLUSTER: {cluster}, CLAIM_TYPE: {claim_type})')
#     plt.xlabel('Calendar Day')
#     plt.ylabel('Feature Value')
#     plt.gca().invert_xaxis()  # Optional: Invert x-axis to show days back from the reference date
    
#     # Add a legend to differentiate the features
#     plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
#     # Show the plot
#     plt.tight_layout()
#     plt.show()
